<a href="https://colab.research.google.com/github/Jack-Chuang/UW-CSE-455/blob/main/final%20project/all_kind_of_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### import , load dataset, and image preprocessing

In [4]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=f10db570b0f138724c36cdf9a41a0d61918dab668cd4c889be184b8a177bf335
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from keras.models import Sequential, load_model
from keras.layers import Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPool2D, DepthwiseConv2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier

from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold, KFold
import pickle
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
from tensorflow.keras import layers

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

import os
import pandas as pd

from keras.datasets import cifar10
# import test, train
from keras.utils import np_utils
import keras.backend as K

# Set seed
from numpy.random import seed
seed(123)

os.environ['PYTHONHASHSEED']=str(123) 

import random
random.seed(123)

import tensorflow as tf
tf.random.set_seed(123)

from sklearn.preprocessing import LabelEncoder

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
    
x_train = X_train.astype('float32')/255
x_test = X_test.astype('float32')/255
y_train = tf.keras.utils.to_categorical(Y_train, 10)
y_test = tf.keras.utils.to_categorical(Y_test, 10)
# y_train_multilabel = tf.keras.utils.to_categorical(Y_train, 10)
# y_test_multilabel = tf.keras.utils.to_categorical(Y_test, 10)
# y_train = LabelEncoder().fit_transform([''.join(str(l)) for l in y_train_multilabel])
# y_test = LabelEncoder().fit_transform([''.join(str(l)) for l in y_test_multilabel])
print(Y_train)
# print(y_train_multilabel)
print(y_train)

170508288/170498071 [==============================] - 11s 0us/step
[[6]
 [9]
 [9]
 ...
 [9]
 [1]
 [1]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [ ]:
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

#### the function of building the baseline neural network

In [ ]:
# Create function
def nn_cl_bo2(cell_magnitude1, cell_magnitude2, conv_filter, conv_kernel, conv_activation, neurons, optimizer, 
              learning_rate, batch_size, epochs, dropout, dropout_rate):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
        
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
     
#     conv_activations = ["relu", "selu"]
        
    cell_magnitude1 = round(cell_magnitude1)
    cell_magnitude2 = round(cell_magnitude2)
    conv_filter = round(conv_filter)
    conv_kernel = round(conv_kernel)
    conv_activ = activationL[round(conv_activation)]
    neurons = round(neurons)
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    
    def SepConv(nn):
        nn.add(DepthwiseConv2D(kernel_size=conv_kernel,
               padding='same',
               depth_multiplier=1,
               strides=1,
               activation=conv_activ,
               use_bias=False))
        nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
        nn.add(layers.ReLU(6.))
        nn.add(Conv2D(conv_filter, kernel_size=conv_kernel, padding='same', activation=conv_activ, use_bias=False, 
                    strides=1))
        nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
        return nn
    
    def MBConv(nn):
        nn.add(DepthwiseConv2D(kernel_size=conv_kernel,
               padding='same',
               depth_multiplier=1,
               strides=1,
               activation=conv_activ,
               use_bias=False))
        nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
        nn.add(layers.ReLU(6.))
        nn.add(DepthwiseConv2D(kernel_size=conv_kernel,
               padding='same',
               depth_multiplier=1,
               strides=1,
               activation=conv_activ,
               use_bias=False))
        nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
        nn.add(layers.ReLU(6.))
        nn.add(Conv2D(conv_filter, kernel_size=conv_kernel, padding='same', activation=conv_activ, use_bias=False, 
                    strides=1))
        nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
        return nn
        
    def nn_cl_fun():
        nn = Sequential()
        nn.add(Conv2D(conv_filter, kernel_size=conv_kernel, input_shape=(32, 32, 3), strides=1, activation=conv_activ, padding='same'))
        nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
        nn.add(layers.ReLU(6.))
        nn = SepConv(nn)
        for i in range(cell_magnitude1):
            nn = MBConv(nn)
        if dropout > 0.5:
            nn.add(Dropout(dropout_rate, seed=123))
        for i in range(cell_magnitude2):
            nn = MBConv(nn)
        nn.add(MaxPool2D(pool_size=(2, 2), padding='same', strides=1))
        nn.add(Flatten())
        nn.add(Dense(10, activation='softmax'))
        nn.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
        return nn
        
    model = nn_cl_fun() 
    model.predict(x_train)
    trainableParams = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
    nonTrainableParams = np.sum([np.prod(v.get_shape()) for v in model.non_trainable_weights])
    totalParams = trainableParams + nonTrainableParams    
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, Y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    # score = cross_val_score(nn, x_train, y_train, scoring=score_acc, fit_params={'callbacks':[es]}).mean()
    # final_score = accuracy * trainableParams * 1/10000
    rounded_score = float("{0:.5f}".format(score))
    
    return score

In [ ]:
def nn_cl_fun_try():
    nn = Sequential()
    nn.add(Conv2D(4, kernel_size=1, input_shape=(32, 32, 3), strides=1, activation='relu', padding='same'))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    nn.add(layers.ReLU(6.))
    nn.add(DepthwiseConv2D(kernel_size=1,
        padding='same',
        depth_multiplier=1,
        strides=1,
        activation='relu',
        use_bias=False))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    nn.add(layers.ReLU(6.))
    nn.add(Conv2D(4, kernel_size=1, padding='same', activation='relu', use_bias=False, 
            strides=1))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))

    nn.add(DepthwiseConv2D(kernel_size=1,
            padding='same',
            depth_multiplier=1,
            strides=1,
            activation='relu',
            use_bias=False))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    nn.add(layers.ReLU(6.))
    nn.add(DepthwiseConv2D(kernel_size=1,
            padding='same',
            depth_multiplier=1,
            strides=1,
            activation='relu',
            use_bias=False))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    nn.add(layers.ReLU(6.))
    nn.add(Conv2D(4, kernel_size=1, padding='same', activation='relu', use_bias=False, 
                strides=1))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))

    nn.add(Dropout(0.2, seed=123))

    nn.add(DepthwiseConv2D(kernel_size=1,
            padding='same',
            depth_multiplier=1,
            strides=1,
            activation='relu',
            use_bias=False))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    nn.add(layers.ReLU(6.))
    nn.add(DepthwiseConv2D(kernel_size=1,
            padding='same',
            depth_multiplier=1,
            strides=1,
            activation='relu',
            use_bias=False))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    nn.add(layers.ReLU(6.))
    nn.add(Conv2D(4, kernel_size=1, padding='same', activation='relu', use_bias=False, 
                strides=1))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))

    nn.add(MaxPool2D(pool_size=(2, 2), padding='same', strides=1))
    nn.add(Flatten())
    nn.add(Dense(10, activation='softmax'))
    nn.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

    return nn

def nn_cl_fun2():
    nn2 = Sequential()
    nn2.add(Conv2D(filters=32, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
    nn2.add(MaxPool2D(pool_size=2))
    nn2.add(Conv2D(filters=32, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
    nn2.add(MaxPool2D(pool_size=2))
    nn2.add(Flatten())
    nn2.add(Dense(10, activation='softmax'))
    nn2.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

    return nn2

# a = tf.keras.utils.to_categorical(np.random.randint(0,10,(50000,1)))
nn = KerasClassifier(build_fn=nn_cl_fun, epochs=50, batch_size=500, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
score = cross_val_score(nn, x_train, a, scoring=score_acc, cv=kfold).mean()
# score = cross_val_score(nn, x_train, y_train, scoring=score_acc).mean()
# nn2 = nn_cl_fun2()
# history2 = nn2.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(x_test, y_test),shuffle=True)
# print(history2.history['accuracy'])
# loss, accuracy = model2.evaluate(x_train, y_train, verbose=0)
# print('Accuracy: %f' % (accuracy))
# print('Loss: %f' % (loss))
# model2.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(x_test, y_test),shuffle=True)
print(score)

NameError: ignored

In [ ]:
x = 13942.63765
g = float("{0:.5f}".format(x))
print(g)

13942.63765


In [ ]:
layer = tf.keras.layers.CategoryEncoding(
           num_tokens=4, output_mode="one_hot")
layer([3.3, 1.99, 0.2, 1.1])

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)>

In [ ]:
indices = [0, 2, -1, 1]
depth = 3
tf.one_hot(np.argmax(indices, axis=0), depth,
           on_value=5.0, off_value=0.0,
           axis=-1)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 5., 0.], dtype=float32)>

In [ ]:
a = tf.keras.utils.to_categorical(np.random.randint(0,10,(5,1)) )
print(np.random.randint(0,10,(5,1)) )
print(a)

[[6]
 [5]
 [8]
 [8]
 [7]]
[[0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]


#### search for the best params using bayesian optimization

In [ ]:
# conv_input = [12]
# conv_filters = [4, 8, 12, 16, 20]
# conv_kernels = [1, 3, 5, 7]
# conv_activations = ["relu", "selu"]
# n_epochs = [20, 30, 40, 50]
# n_batch = [1, 150]
# n_diff = [0, 12]

params_nn2 ={
    'cell_magnitude1': (0, 5),
    'cell_magnitude2': (0, 5),
    'conv_filter': ((4, 20)),
    'conv_kernel': ((1, 5)),
    'conv_activation': ((0, 9)),
    'neurons': (10, 100),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}

# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo2, params_nn2, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | batch_... | cell_m... | cell_m... | conv_a... | conv_f... | conv_k... |  dropout  | dropou... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  nan      |  689.7    |  0.8453   |  2.18     |  6.923    |  8.725    |  1.597    |  0.02248  |  0.1261   |  39.09    |  0.3443   |  99.16    |  1.664    |
|  2        |  nan      |  265.0    |  3.348    |  3.106    |  2.468    |  11.46    |  1.473    |  0.07396  |  0.2702   |  83.52    |  0.8422   |  83.37    |  6.937    |


KeyboardInterrupt: ignored

#### collect the best hyperparamters and number of layers

In [ ]:
params_nn_ = nn_bo.max['params']

learning_rate = params_nn_['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['conv_activation'] = activationL[round(params_nn_['conv_activation'])]

params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['cell_magnitude1'] = round(params_nn_['cell_magnitude1'])
params_nn_['cell_magnitude2'] = round(params_nn_['cell_magnitude2'])
params_nn_['neurons'] = round(params_nn_['neurons'])
params_nn_['conv_filter'] = round(params_nn_['conv_filter'])
params_nn_['conv_kernel'] = round(params_nn_['conv_kernel'])
params_nn_['conv_kernel'] = round(params_nn_['conv_kernel'])
# params_nn_['dropout'] = round(params_nn_['dropout'])
# params_nn_['dropout_rate'] = round(params_nn_['dropout_rate'])
# params_nn_['learning_rate'] = round(params_nn_['learning_rate'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','Adam']
optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
             'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
             'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
             'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
params_nn_['optimizer'] = optimizerD[optimizerL[round(params_nn_['optimizer'])]]

params_nn_

{'batch_size': 285,
 'cell_magnitude1': 2,
 'cell_magnitude2': 4,
 'conv_activation': 'sigmoid',
 'conv_filter': 20,
 'conv_kernel': 2,
 'dropout': 0.7214115754924483,
 'dropout_rate': 0.1982676082332063,
 'epochs': 26,
 'learning_rate': 0.7003114523771421,
 'neurons': 34,
 'optimizer': <keras.optimizer_v2.nadam.Nadam at 0x7efdb90bc750>}

#### use the collection of the best params to rebuild the model and try it on the dataset

In [ ]:
# Fitting Neural Network
def SepConv_final(nn):
    nn.add(DepthwiseConv2D(kernel_size=params_nn_['conv_kernel'],
            padding='same',
            depth_multiplier=1,
            strides=1,
            activation=params_nn_['conv_activation'],
            use_bias=False))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    nn.add(layers.ReLU(6.))
    nn.add(Conv2D(params_nn_['conv_filter'], kernel_size=params_nn_['conv_kernel'], padding='same', activation=params_nn_['conv_activation'], use_bias=False, 
                strides=1))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    return nn

def MBConv_final(nn):
    nn.add(DepthwiseConv2D(kernel_size=params_nn_['conv_kernel'],
            padding='same',
            depth_multiplier=1,
            strides=1,
            activation=params_nn_['conv_activation'],
            use_bias=False))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    nn.add(layers.ReLU(6.))
    nn.add(DepthwiseConv2D(kernel_size=params_nn_['conv_kernel'],
            padding='same',
            depth_multiplier=1,
            strides=1,
            activation=params_nn_['conv_activation'],
            use_bias=False))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    nn.add(layers.ReLU(6.))
    nn.add(Conv2D(params_nn_['conv_filter'], kernel_size=params_nn_['conv_kernel'], padding='same', activation=params_nn_['conv_activation'], use_bias=False, 
                strides=1))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    return nn
    
def nn_cl_fun_final():
    nn = Sequential()
    nn.add(Conv2D(params_nn_['conv_filter'], kernel_size=params_nn_['conv_kernel'], input_shape=(32, 32, 3), strides=1, activation=params_nn_['conv_activation'], padding='same'))
    nn.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
    nn.add(layers.ReLU(6.))
    nn = SepConv_final(nn)
    for i in range(params_nn_['cell_magnitude1']):
        nn = MBConv_final(nn)
    if params_nn_['dropout'] > 0.5:
        nn.add(Dropout(params_nn_['dropout_rate'], seed=123))
    for i in range(params_nn_['cell_magnitude2']):
        nn = MBConv_final(nn)
    nn.add(MaxPool2D(pool_size=(2, 2), padding='same', strides=1))
    nn.add(Flatten())
    nn.add(Dense(10, activation='softmax'))
    nn.compile(loss="categorical_crossentropy", optimizer=params_nn_['optimizer'], metrics=['accuracy'])
    return nn
        
# es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
# nn = KerasClassifier(build_fn=nn_cl_fun, epochs=params_nn_['epochs'], batch_size=params_nn_['batch_size'],
#                          verbose=0)
model_final = nn_cl_fun()
loss, accuracy = model_final.evaluate(x_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy))
print('Loss: %f' % (loss))

313/313 [==============================] - 2s 4ms/step - loss: 2.3969 - accuracy: 0.1000
Accuracy: 0.100000
Loss: 2.396929


In [ ]:
model6 = Sequential()
model6.add(Conv2D(filters=84, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
model6.add(MaxPool2D(pool_size=2))
model6.add(Conv2D(filters=84, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
model6.add(MaxPool2D(pool_size=2))
model6.add(Conv2D(filters=84, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
model6.add(MaxPool2D(pool_size=2))
model6.add(Conv2D(filters=84, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
model6.add(MaxPool2D(pool_size=2))
model6.add(Conv2D(filters=84, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
#model6.add(MaxPool2D(pool_size=2))
model6.add(Conv2D(filters=84, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
model6.add(MaxPool2D(pool_size=2))
model6.add(Flatten())
model6.add(Dense(10, activation='softmax'))
model6.summary()

model6.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model6.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(x_test, y_test),shuffle=True)

Model: "sequential_85"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_566 (Conv2D)         (None, 32, 32, 84)        2352      
                                                                 
 max_pooling2d_87 (MaxPoolin  (None, 16, 16, 84)       0         
 g2D)                                                            
                                                                 
 conv2d_567 (Conv2D)         (None, 16, 16, 84)        63588     
                                                                 
 max_pooling2d_88 (MaxPoolin  (None, 8, 8, 84)         0         
 g2D)                                                            
                                                                 
 conv2d_568 (Conv2D)         (None, 8, 8, 84)          63588     
                                                                 
 max_pooling2d_89 (MaxPoolin  (None, 4, 4, 84)       

In [ ]:
# baseline model with dropout and data augmentation on the cifar10 dataset
import sys
import tensorflow as tf
from matplotlib import pyplot
from keras.datasets import cifar10
# from keras.utils import to_categorical
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Flatten
# from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras import regularizers
from keras import Model
from keras.activations import relu
from keras import layers as Layers 

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = np_utils.to_categorical(trainY)
	testY = np_utils.to_categorical(testY)
	return trainX, trainY, testX, testY
 
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return model

 
# define cnn model
def define_model():

	weight_decay = 0.001

	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3), kernel_regularizer=regularizers.l2(weight_decay)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
	model.add(BatchNormalization())
  # add residual
	identity_block(model, 64)
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))	
  # add residual
	identity_block(model, 128)
	model.add(GlobalAveragePooling2D())
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = tf.keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
 
# run the test harness for evaluating a model
def run_test_harness():
  # load dataset
  trainX, trainY, testX, testY = load_dataset()
  # prepare pixel data
  trainX, testX = prep_pixels(trainX, testX)
  # define model
  model = define_model()
  # evaluate model
  history = model.fit(trainX, trainY, epochs=100, batch_size=64, verbose=1, validation_data=(testX, testY),shuffle=True)
  # loss, acc = model.evaluate(testX, testY, verbose=0)
  # print('> %.3f' % (acc * 100.0))
  # learning curves
  summarize_diagnostics(history)
 
# entry point, run the test harness
run_test_harness()

TypeError: ignored

data augmentation

In [4]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.ndimage import rotate
sns.set(color_codes=True)


def translate(img, shift, dir, r):
    directions = ['right', 'left', 'down', 'up']
    rolls = [True, False]
    direction = directions[dir]
    roll = rolls[r]
    img = img.copy()
    if direction == 'right':
        right_slice = img[:, -shift:].copy()
        img[:, shift:] = img[:, :-shift]
        if roll:
            img[:,:shift] = np.fliplr(right_slice)
    if direction == 'left':
        left_slice = img[:, :shift].copy()
        img[:, :-shift] = img[:, shift:]
        if roll:
            img[:, -shift:] = left_slice
    if direction == 'down':
        down_slice = img[-shift:, :].copy()
        img[shift:, :] = img[:-shift,:]
        if roll:
            img[:shift, :] = down_slice
    if direction == 'up':
        upper_slice = img[:shift, :].copy()
        img[:-shift, :] = img[shift:, :]
        if roll:
            img[-shift:,:] = upper_slice
    return img

def random_crop(img, crop_size):
    assert crop_size[0] <= img.shape[0] and crop_size[1] <= img.shape[1], "Crop size should be less than image size"
    img = img.copy()
    w, h = img.shape[:2]
    x, y = np.random.randint(h-crop_size[0]), np.random.randint(w-crop_size[1])
    img = img[y:y+crop_size[0], x:x+crop_size[1]]
    return img

def rotate_img(img, angle, bg_patch):
    assert len(img.shape) <= 3, "Incorrect image shape"
    rgb = len(img.shape) == 3
    if rgb:
        bg_color = np.mean(img[:bg_patch[0], :bg_patch[1], :], axis=(0,1))
    else:
        bg_color = np.mean(img[:bg_patch[0], :bg_patch[1]])
    img = rotate(img, angle, reshape=False)
    mask = [img <= 0, np.any(img <= 0, axis=-1)][rgb]
    img[mask] = bg_color
    return img

def gaussian_noise(img, mean, sigma):
    img = img.copy()
    noise = np.random.normal(mean, sigma, img.shape)
    mask_overflow_upper = img+noise >= 1.0
    mask_overflow_lower = img+noise < 0
    noise[mask_overflow_upper] = 1.0
    noise[mask_overflow_lower] = 0
    img += noise
    return img


def distort(img, ori, x_scale, y_scale, func):
    orientations = ['hor', 'ver'], "dist_orient should be 'horizontal'|'vertical'"
    functions = [np.sin, np.cos], "supported functions are np.sin and np.cos"
    orientation = orientations[ori]
    function = functions[func]
    assert 0.00 <= x_scale <= 0.1, "x_scale should be in [0.0, 0.1]"
    assert 0 <= y_scale <= min(img.shape[0], img.shape[1]), "y_scale should be less then image size"
    img_dist = img.copy()
    
    def shift(x):
        return int(y_scale * function(np.pi * x * x_scale))
    
    for c in range(3):
        for i in range(img.shape[orientation.startswith('ver')]):
            if orientation.startswith('ver'):
                img_dist[:, i, c] = np.roll(img[:, i, c], shift(i))
            else:
                img_dist[i, :, c] = np.roll(img[i, :, c], shift(i))
            
    return img_dist


def change_channel_ratio(img, c, ratio):
    channels = ['r', 'g', 'b']
    channel = channels[c]
    img = img.copy()
    ci = 'rgb'.index(channel)
    img[:, :, ci] *= ratio
    return img


def change_channel_ratio_gauss(img, c, mean, sigma):
    channels = ['r', 'g', 'b']
    channel = channels[c]
    img = img.copy()
    ci = 'rgb'.index(channel)
    img[:, :, ci] = gaussian_noise(img[:, :, ci], mean=mean, sigma=sigma)
    return img

def cutoff(img, width, height):
  img_new = img.copy()
  w, h = img.shape[:2]
  x, y = np.random.randint(h), np.random.randint(w)
  img_new[y:y+width, x:x+height] = 0
  return img_new


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


data processing

In [5]:
# baseline model with dropout and data augmentation on the cifar10 dataset
import sys
import tensorflow as tf
from matplotlib import pyplot
from keras.datasets import cifar10
# from keras.utils import to_categorical
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Flatten
# from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras import regularizers
from keras import Model
from keras.activations import relu
from keras import layers as Layers 
from keras.regularizers import l2
import h5py
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import keras as k
import cv2

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = np_utils.to_categorical(trainY)
	testY = np_utils.to_categorical(testY)
	return trainX, trainY, testX, testY
 
# scale pixels
def prep_pixels(train, test):
  # convert from integers to floats
  train_norm = train.astype('float32')
  test_norm = test.astype('float32')
  # # normalize to range 0-1
  # train_norm = train_norm / 255.0
  # test_norm = test_norm / 255.0
  #z-score
  mean = np.mean(trainX,axis=(0,1,2,3))
  std = np.std(trainX,axis=(0,1,2,3))
  x_train = (trainX-mean)/(std+1e-7)
  x_test = (testX-mean)/(std+1e-7)
  # return normalized images
  return train_norm, test_norm

# load dataset
trainX, trainY, testX, testY = load_dataset()

# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)

# set up image augmentation
# datagen = ImageDataGenerator(
#     featurewise_center=True, samplewise_center=True,
#     featurewise_std_normalization=True, samplewise_std_normalization=True,
#     rotation_range=90, width_shift_range=0.2,
#     height_shift_range=0.2, brightness_range=None, shear_range=0.2, zoom_range=0.2,
#     channel_shift_range=0.2, fill_mode='nearest', cval=0.2,
#     horizontal_flip=True, vertical_flip=True, rescale=1.2
# )
# datagen.fit(trainX)
x_train = trainX
y_train = trainY

count = 0
for img in trainX[0:50000]:
  imgs_distorted = []
  for ori in [0, 1]:
    for x_param in [0.01, 0.02, 0.03, 0.04]:
      for y_param in [2, 4, 6, 8, 10]:
          imgs_distorted.append(distort(img, ori=ori, x_scale=x_param, y_scale=y_param, func=np.random.randint(0,1)))
  imgs_translate = translate(img, shift=np.random.randint(1,10), dir=np.random.randint(0,3), r=np.random.randint(0,1))
  imgs_crop = random_crop(img, crop_size=(np.random.randint(1,10), np.random.randint(1,10)))
  imgs_crop_resized = cv2.resize(imgs_crop, dsize=(32, 32), interpolation=cv2. INTER_CUBIC)
  imgs_rotate = rotate_img(img, angle=np.random.randint(10,30), bg_patch=(np.random.randint(1,5),np.random.randint(1,5)))
  imgs_gaussian = gaussian_noise(img, mean=0, sigma=np.random.random_sample())
  imgs_change_channel_ratio = change_channel_ratio(img, c=np.random.randint(0,2), ratio=np.random.random_sample())
  imgs_change_channel_ratio_gauss = change_channel_ratio_gauss(img, c=np.random.randint(0,2), mean=0, sigma=np.random.random_sample())
  img_cutoff = cutoff(img, width=np.random.randint(3,12), height=np.random.randint(3,12))
  x_train = np.append(x_train, imgs_distorted, axis=0)
  for i in range(len(imgs_distorted)):
    y_train = np.append(y_train, [trainY[count]], axis=0)
  x_train = np.append(x_train, [imgs_translate], axis=0)
  for i in range(len([imgs_translate])):
    y_train = np.append(y_train, [trainY[count]], axis=0)
  x_train = np.append(x_train, [imgs_crop_resized], axis=0)
  for i in range(len([imgs_crop_resized])):
    y_train = np.append(y_train, [trainY[count]], axis=0)
  x_train = np.append(x_train, [imgs_rotate], axis=0)
  for i in range(len([imgs_rotate])):
    y_train = np.append(y_train, [trainY[count]], axis=0)
  x_train = np.append(x_train, [imgs_gaussian], axis=0)
  for i in range(len([imgs_gaussian])):
    y_train = np.append(y_train, [trainY[count]], axis=0)
  x_train = np.append(x_train, [imgs_change_channel_ratio], axis=0)
  for i in range(len([imgs_change_channel_ratio])):
    y_train = np.append(y_train, [trainY[count]], axis=0)
  x_train = np.append(x_train, [imgs_change_channel_ratio_gauss], axis=0)
  for i in range(len([imgs_change_channel_ratio_gauss])):
    y_train = np.append(y_train, [trainY[count]], axis=0)
  x_train = np.append(x_train, [img_cutoff], axis=0)
  for i in range(len([img_cutoff])):
    y_train = np.append(y_train, [trainY[count]], axis=0)
  count += 1
  print(count)

AttributeError: ignored

using only cutoff

In [58]:
x_train_rest = x_train
y_train_rest = y_train

count = 1000
for img in trainX[1001:50000]:
  # imgs_distorted = []
  # for ori in [0, 1]:
  #   for x_param in [0.01, 0.02, 0.03, 0.04]:
  #     for y_param in [2, 4, 6, 8, 10]:
  #         imgs_distorted.append(distort(img, ori=ori, x_scale=x_param, y_scale=y_param, func=np.random.randint(0,1)))
  imgs_translate = translate(img, shift=np.random.randint(1,5), dir=np.random.randint(0,3), r=np.random.randint(0,1))
  imgs_crop = random_crop(img, crop_size=(np.random.randint(15,25), np.random.randint(15,25)))
  imgs_crop_resized = cv2.resize(imgs_crop, dsize=(32, 32), interpolation=cv2. INTER_CUBIC)
  imgs_rotate = rotate_img(img, angle=np.random.randint(5,20), bg_patch=(np.random.randint(1,5),np.random.randint(1,5)))
  imgs_gaussian = gaussian_noise(img, mean=0, sigma=np.random.random_sample())
  imgs_change_channel_ratio = change_channel_ratio(img, c=np.random.randint(0,2), ratio=np.random.random_sample())
  imgs_change_channel_ratio_gauss = change_channel_ratio_gauss(img, c=np.random.randint(0,2), mean=0, sigma=np.random.random_sample())
  img_cutoff = cutoff(img, width=np.random.randint(3,10), height=np.random.randint(3,10))
  # x_train_rest = np.append(x_train_rest, imgs_distorted, axis=0)
  # for i in range(len(imgs_distorted)):
  #   y_train_rest = np.append(y_train_rest, [trainY[count]], axis=0)
  x_train_rest = np.append(x_train_rest, [imgs_translate], axis=0)
  for i in range(len([imgs_translate])):
    y_train_rest = np.append(y_train_rest, [trainY[count]], axis=0)
  x_train_rest = np.append(x_train_rest, [imgs_crop_resized], axis=0)
  for i in range(len([imgs_crop_resized])):
    y_train_rest = np.append(y_train_rest, [trainY[count]], axis=0)
  x_train_rest = np.append(x_train_rest, [imgs_rotate], axis=0)
  for i in range(len([imgs_rotate])):
    y_train_rest = np.append(y_train_rest, [trainY[count]], axis=0)
  x_train_rest = np.append(x_train_rest, [imgs_gaussian], axis=0)
  for i in range(len([imgs_gaussian])):
    y_train_rest = np.append(y_train_rest, [trainY[count]], axis=0)
  x_train_rest = np.append(x_train_rest, [imgs_change_channel_ratio], axis=0)
  for i in range(len([imgs_change_channel_ratio])):
    y_train_rest = np.append(y_train_rest, [trainY[count]], axis=0)
  x_train_rest = np.append(x_train_rest, [imgs_change_channel_ratio_gauss], axis=0)
  for i in range(len([imgs_change_channel_ratio_gauss])):
    y_train_rest = np.append(y_train_rest, [trainY[count]], axis=0)
  x_train_rest = np.append(x_train_rest, [img_cutoff], axis=0)
  for i in range(len([img_cutoff])):
    y_train_rest = np.append(y_train_rest, [trainY[count]], axis=0)
  count += 1
  print(count)

1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011


KeyboardInterrupt: ignored

In [6]:
# baseline model with dropout and data augmentation on the cifar10 dataset
import sys
import tensorflow as tf
from matplotlib import pyplot
from keras.datasets import cifar10
# from keras.utils import to_categorical
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Flatten
# from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras import regularizers
from keras import Model
from keras.activations import relu
from keras import layers as Layers 
from keras.regularizers import l2
import h5py
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import keras as k
import cv2

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = np_utils.to_categorical(trainY)
	testY = np_utils.to_categorical(testY)
	return trainX, trainY, testX, testY
 
# scale pixels
def prep_pixels(train, test):
  # convert from integers to floats
  train_norm = train.astype('float32')
  test_norm = test.astype('float32')
  # # normalize to range 0-1
  # train_norm = train_norm / 255.0
  # test_norm = test_norm / 255.0
  #z-score
  mean = np.mean(trainX,axis=(0,1,2,3))
  std = np.std(trainX,axis=(0,1,2,3))
  x_train = (trainX-mean)/(std+1e-7)
  x_test = (testX-mean)/(std+1e-7)
  # return normalized images
  return train_norm, test_norm

# load dataset
trainX, trainY, testX, testY = load_dataset()

# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)

# set up image augmentation
# datagen = ImageDataGenerator(
#     featurewise_center=True, samplewise_center=True,
#     featurewise_std_normalization=True, samplewise_std_normalization=True,
#     rotation_range=90, width_shift_range=0.2,
#     height_shift_range=0.2, brightness_range=None, shear_range=0.2, zoom_range=0.2,
#     channel_shift_range=0.2, fill_mode='nearest', cval=0.2,
#     horizontal_flip=True, vertical_flip=True, rescale=1.2
# )
# datagen.fit(trainX)
x_train = trainX
y_train = trainY

count = 0
for img in trainX[0:50000]:
  img_cutoff = cutoff(img, width=np.random.randint(1,12), height=np.random.randint(1,12))
  imgs_crop = random_crop(img, crop_size=(np.random.randint(15,25), np.random.randint(15,25)))
  imgs_crop_resized = cv2.resize(imgs_crop, dsize=(32, 32), interpolation=cv2. INTER_CUBIC)
  imgs_change_channel_ratio_gauss = change_channel_ratio_gauss(img, c=np.random.randint(0,2), mean=0, sigma=np.random.random_sample())
  x_train = np.append(x_train, [img_cutoff], axis=0)
  for i in range(len([img_cutoff])):
    y_train = np.append(y_train, [trainY[count]], axis=0)
  x_train = np.append(x_train, [imgs_crop_resized], axis=0)
  for i in range(len([imgs_crop_resized])):
    y_train = np.append(y_train, [trainY[count]], axis=0)
  x_train = np.append(x_train, [imgs_change_channel_ratio_gauss], axis=0)
  for i in range(len([imgs_change_channel_ratio_gauss])):
    y_train = np.append(y_train, [trainY[count]], axis=0)
  count += 1
  print(count)

Streaming output truncated to the last 5000 lines.
45001
45002
45003
45004
45005
45006
45007
45008
45009
45010
45011
45012
45013
45014
45015
45016
45017
45018
45019
45020
45021
45022
45023
45024
45025
45026
45027
45028
45029
45030
45031
45032
45033
45034
45035
45036
45037
45038
45039
45040
45041
45042
45043
45044
45045
45046
45047
45048
45049
45050
45051
45052
45053
45054
45055
45056
45057
45058
45059
45060
45061
45062
45063
45064
45065
45066
45067
45068
45069
45070
45071
45072
45073
45074
45075
45076
45077
45078
45079
45080
45081
45082
45083
45084
45085
45086
45087
45088
45089
45090
45091
45092
45093
45094
45095
45096
45097
45098
45099
45100
45101
45102
45103
45104
45105
45106
45107
45108
45109
45110
45111
45112
45113
45114
45115
45116
45117
45118
45119
45120
45121
45122
45123
45124
45125
45126
45127
45128
45129
45130
45131
45132
45133
45134
45135
45136
45137
45138
45139
45140
45141
45142
45143
45144
45145
45146
45147
45148
45149
45150
45151
45152
45153
45154
45155
45156
45157
45158
4

In [25]:
from keras.layers import ReLU
############################################### build model: VGG 19 ###############################################################################
weight_decay = 5e-4

model_VGG19 = Sequential()

model_VGG19.add(Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=(32, 32, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())
model_VGG19.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())

model_VGG19.add(MaxPooling2D(pool_size=2))

model_VGG19.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())
model_VGG19.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())

model_VGG19.add(MaxPooling2D(pool_size=2))

model_VGG19.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())
model_VGG19.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())
model_VGG19.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())
model_VGG19.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())

model_VGG19.add(MaxPooling2D(pool_size=2))

model_VGG19.add(Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())
model_VGG19.add(Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())
model_VGG19.add(Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())
model_VGG19.add(Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())

model_VGG19.add(MaxPooling2D(pool_size=2))

model_VGG19.add(Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())
model_VGG19.add(Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())
model_VGG19.add(Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())
model_VGG19.add(Conv2D(filters=512, kernel_size=3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model_VGG19.add(BatchNormalization(epsilon=1e-3, momentum=0.999))
model_VGG19.add(ReLU())

model_VGG19.add(MaxPooling2D(pool_size=2))

model_VGG19.add(Flatten())
model_VGG19.add(Dense(10, activation='softmax'))

model_VGG19.summary()

learning_rate = 0.1
epochs = 50
decay_rate = learning_rate / epochs
opt = tf.keras.optimizers.SGD(learning_rate=learning_rate , decay=decay_rate, momentum=0.9)
model_VGG19.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

history = model_VGG19.fit(trainX, trainY, epochs=epochs, batch_size=128, verbose=1, validation_split=0.1, shuffle=True)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_245 (Conv2D)         (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization_244 (Ba  (None, 32, 32, 64)       256       
 tchNormalization)                                               
                                                                 
 re_lu_112 (ReLU)            (None, 32, 32, 64)        0         
                                                                 
 conv2d_246 (Conv2D)         (None, 32, 32, 64)        36928     
                                                                 
 batch_normalization_245 (Ba  (None, 32, 32, 64)       256       
 tchNormalization)                                               
                                                                 
 re_lu_113 (ReLU)            (None, 32, 32, 64)       

In [26]:
model_VGG19.evaluate(testX, testY, batch_size=64, verbose=1)

157/157 [==============================] - 2s 13ms/step - loss: 8.8335 - accuracy: 0.3755


[8.833462715148926, 0.37549999356269836]

In [1]:
def identity_block(x, filter, weight_decay):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), kernel_initializer='he_uniform', padding = 'same', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), kernel_initializer='he_uniform', padding = 'same', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

##################################################### build model: ResNet 34 #####################################################################################
inputs = tf.keras.Input(shape=(32, 32, 3))
weight_decay = 5e-4

x = Conv2D(64, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(inputs)
x = BatchNormalization(epsilon=1e-3, momentum=0.999)(x) 
# x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(x)
# x = BatchNormalization(epsilon=1e-3, momentum=0.999)(x) 

x = MaxPooling2D(2, 2)(x)
x = Dropout(0.2)(x)

# add residual
x = identity_block(x, 64, weight_decay)
x = identity_block(x, 64, weight_decay)
x = identity_block(x, 64, weight_decay)

x = Conv2D(128, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(x)
x = BatchNormalization(epsilon=1e-3, momentum=0.999)(x) 
x = Conv2D(128, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(x)
x = BatchNormalization(epsilon=1e-3, momentum=0.999)(x) 
x = identity_block(x, 128, weight_decay)
x = identity_block(x, 128, weight_decay)
x = identity_block(x, 128, weight_decay)

x = Conv2D(256, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(x)
x = BatchNormalization(epsilon=1e-3, momentum=0.999)(x) 
x = Conv2D(256, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(x)
x = BatchNormalization(epsilon=1e-3, momentum=0.999)(x) 
x = identity_block(x, 256, weight_decay)
x = identity_block(x, 256, weight_decay)
x = identity_block(x, 256, weight_decay)
x = identity_block(x, 256, weight_decay)
x = identity_block(x, 256, weight_decay)

x = Conv2D(512, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(x)
x = BatchNormalization(epsilon=1e-3, momentum=0.999)(x) 
x = Conv2D(512, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(x)
x = BatchNormalization(epsilon=1e-3, momentum=0.999)(x) 
x = identity_block(x, 512, weight_decay)
x = identity_block(x, 512, weight_decay)

x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(10, activation='softmax')(x)
##################################################### build model #####################################################################################

# compile model
model_ResNet34 = tf.keras.Model(inputs=inputs, outputs=outputs)
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=0.1,
#     decay_steps=10000,
#     decay_rate=1.1)
model_ResNet34.summary()
learning_rate = 0.1
epochs = 30
decay_rate = learning_rate / epochs
opt = tf.keras.optimizers.SGD(learning_rate=learning_rate , decay=decay_rate, momentum=0.9)
model_ResNet34.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# evaluate model
# history = model.fit(x_train, y_train, epochs=epochs, batch_size=128, verbose=1, validation_data=(testX, testY), shuffle=True)
history = model_ResNet34.fit(trainX, trainY, epochs=epochs, batch_size=128, verbose=1, validation_split=0.1, shuffle=True)

NameError: ignored

In [ ]:
model_ResNet34.evaluate(testX, testY, batch_size=64, verbose=1)

In [ ]:
while True:
  print("Fuck")